In [ ]:
# Importamos bibliotecas.
using Plots, DelimitedFiles, LaTeXStrings

# NOTA: El artículo citado como 'Daka' es:
# Daka, E., Phan, N. N., & Kain, B. (2019). 
# Perturbing the ground state of Dirac stars. 
# Physical Review D, 100(8), 084042.

In [ ]:
# Calculamos la condición inicial (estados estáticos).

# Definimos el sistema diferencial en sí.
function rhsStatic(w, r, m, σ, f, g)

    # Definimos funciones auxiliares (son las ecuaciones (52) de Daka).
    ene = 1 - (2*m)/(r)
    rho = ( (2*w)/(r^2 * σ) )*(f^2 + g^2)
    srr = ( 2*ene/r^2)*( w*(f^2 + g^2)/(ene*σ) - (2*f*g)/(r*sqrt(ene)) - (f^2 - g^2)/sqrt(ene) )
    zet = ( r/ene)*(rho - srr) - (2*m)/(r^2 * ene)

    # Definimos las ecuaciones diferenciales de los estados estáticos ((51) de Daka).
    dm = r^2 * rho
    dσ = (σ*r/ene)*(rho + srr)
    df = f*( zet/2 + 1/(r*sqrt(ene)) ) - g*(1/sqrt(ene) + w/(ene*σ))
    dg = g*( zet/2 - 1/(r*sqrt(ene)) ) - f*(1/sqrt(ene) - w/(ene*σ))

    return [dm, dσ, df, dg]
end

# Definimos una función que calcule haga rk4 (radial) para calcular el siguiente paso.
function rk4Static(dr, w, r, m, σ, f, g)

    k1  = rhsStatic(w, r, m, σ, f, g)
    k1m = k1[1]
    k1σ = k1[2]
    k1f = k1[3]
    k1g = k1[4]

    k2  = rhsStatic(w, r + dr/2, m + (k1m*dr/2), σ + (k1σ*dr/2), f + (k1f*dr/2), g + (k1g*dr/2))
    k2m = k2[1]
    k2σ = k2[2]
    k2f = k2[3]
    k2g = k2[4]

    k3  = rhsStatic(w, r + dr/2, m + (k2m*dr/2), σ + (k2σ*dr/2), f + (k2f*dr/2), g + (k2g*dr/2))
    k3m = k3[1]
    k3σ = k3[2]
    k3f = k3[3]
    k3g = k3[4]

    k4  = rhsStatic(w, r + dr, m + k3m*dr, σ + k3σ*dr, f + k3f*dr, g + k3g*dr)
    k4m = k4[1]
    k4σ = k4[2]
    k4f = k4[3]
    k4g = k4[4]

    rrk4 = r + dr
    mrk4 = m + (k1m + 2*k2m + 2*k3m + k4m)*(dr/6)
    σrk4 = σ + (k1σ + 2*k2σ + 2*k3σ + k4σ)*(dr/6)
    frk4 = f + (k1f + 2*k2f + 2*k3f + k4f)*(dr/6)
    grk4 = g + (k1g + 2*k2g + 2*k3g + k4g)*(dr/6)
    
    return [rrk4, mrk4, σrk4, frk4, grk4]
end

function solver(fi, w, Nr, dr)
    # Definimos las condiciones iniciales ((55) de Daka).
    r0 = dr
    m0 = (2/3)*(fi^2)*(dr^3)*w
    σ0 = 1 + (1/3)*(fi^2)*(4*w - 1)*(dr^2)
    f0 = fi*dr
    g0 = (1/3)*fi*(w - 1)*(dr^2)

    # Definimos los arreglos a usar.
    r = zeros(Nr)
    m = zeros(Nr)
    σ = zeros(Nr)
    f = zeros(Nr)
    g = zeros(Nr)

    # Llenamos las condiciones iniciales (y las simetrías).
    r[2] = r0
    m[2] = m0
    σ[2] = σ0
    f[2] = f0
    g[2] = g0

    r[1] = -r0
    m[1] = -m0
    σ[1] = +σ0
    f[1] = -f0
    g[1] = +g0

    # Comenzamos a solucionar en sí.
    for j in 2:Nr-1
        soluto = rk4Static(dr, w, r[j], m[j], σ[j], f[j], g[j])

        r[j+1] = soluto[1]
        m[j+1] = soluto[2]
        σ[j+1] = soluto[3]
        f[j+1] = soluto[4]
        g[j+1] = soluto[5]
    end

    counter = 0

    return [counter, w, r, m, σ, f, g]
end

function shooting(fi, w, Nr, dr)
    
    # Definimos la primer diferencial de 'w'.
    tolerancia = dr^4
    dw = 0.00001

    # Definimos un contador para ver cuántas veces necesitamos solucionar.
    counter = 0
    
    # Solucionamos por primera vez.
    f = solver(fi, w, Nr, dr)[6]
    counter += 1

    while abs( f[end] ) > tolerancia
        
        if f[end] > tolerancia
            # Actualizamos la 'w'.
            w = w + dw

            # Volvemos a solucionar.
            f = solver(fi, w, Nr, dr)[6]

        else
            # Aquí la 'w' ha sido muy grande, entonces debemos regresar.
            w = w - dw

            # Ahora hacemos más pequeña la varaición y volvemos a avanzar.
            dw = dw/2
            w = w + dw

            # Volvemos a solucionar.
            f = solver(fi, w, Nr, dr)[6]
        end
        
        counter += 1
    end
        
    # Solucionamos una última vez (pero no la contamos).
    solucion = solver(fi, w, Nr, dr)
    r = solucion[3]
    m = solucion[4]
    σ = solucion[5]
    f = solucion[6]
    g = solucion[7]
    
    return [counter, w, r, m, σ, f, g]
end

# Solucionamos en sí para encontrar la primer línea temporal.
function initial(fi, w, Nr, dr)

    # Aplicamos el shooting.
    println("Comenzamos con el shooting...")
    soluto = shooting(fi, w, Nr, dr)
    counter = soluto[1]
    w = soluto[2]
    r = soluto[3]
    m = soluto[4]
    σ = soluto[5]
    f = soluto[6]
    g = soluto[7]
    println("Iteraciones del shooting: $counter \n")

    # Calculamos la parte real e imaginaria de las funciones.
    f1 = copy(f)
    f2 = zeros(Nr)
    g1 = zeros(Nr)
    g2 = copy(g)
    
    # Regresamos una adimensión (ver el reporte del Servicio Social).
    σ00 = 1/σ[Nr]

    # Calculamos los coeficientes métricos.
    a = 1 ./ sqrt.( 1 .- 2 .* (m ./ r) )
    α =  σ00 .* σ .* sqrt.( 1 .- 2 .* (m ./ r) )
    σ = a .* α
    
    return [r, m, a, α, σ, f1, f2, g1, g2]
end

In [ ]:
# Definimos una función para los rhs dinámicos.
function rhs(Nr, dr, r, a, α, f1, f2, g1, g2)

    # Definimos arreglos para guardar las fuentes, o sea, 
    # los lados derechos (los right-hand side).
    f1_f = zeros(Nr)
    f2_f = zeros(Nr)
    g1_f = zeros(Nr)
    g2_f = zeros(Nr)

    # Llenamos estos arreglos con (35) de Daka.
    for j in 3:Nr-1
        f1_f[j] = -( α[j]/r[j] )*( g1[j] - r[j]*f2[j] ) - 
                sqrt( α[j]/a[j] )*( 
                sqrt( α[j+1]/a[j+1] )*g1[j+1] -
                sqrt( α[j-1]/a[j-1] )*g1[j-1] )/(2*dr)
        
        f2_f[j] = -( α[j]/r[j] )*( g2[j] + r[j]*f1[j] ) - 
                sqrt( α[j]/a[j] )*( 
                sqrt( α[j+1]/a[j+1] )*g2[j+1] -
                sqrt( α[j-1]/a[j-1] )*g2[j-1] )/(2*dr)
        
        g1_f[j] = ( α[j]/r[j] )*( f1[j] - r[j]*g2[j] ) - 
                sqrt( α[j]/a[j] )*( 
                sqrt( α[j+1]/a[j+1] )*f1[j+1] -
                sqrt( α[j-1]/a[j-1] )*f1[j-1] )/(2*dr)
        
        g2_f[j] = ( α[j]/r[j] )*( f2[j] + r[j]*g1[j] ) - 
                sqrt( α[j]/a[j] )*( 
                sqrt( α[j+1]/a[j+1] )*f2[j+1] -
                sqrt( α[j-1]/a[j-1] )*f2[j-1] )/(2*dr)
    end

    # Como en j=1 tenemos la condición de simetría solo nos interesa
    # calcular las fuentes desde j=2.
    # NOTA: Por alguna razón no funciona definir derivadas hacia adelante en este punto particular.
    f1_f[2] = f1_f[3]
    f2_f[2] = f2_f[3]
    g1_f[2] = g1_f[3]
    g2_f[2] = g2_f[3]

    return [f1_f, f2_f, g1_f, g2_f]
end

In [ ]:
# Definimos una función para solucionar la métrica.

# Las ecuaciones son (36) de Daka con las convenciones de (37).
# Además solucionaremos con rk2 combinado con el método 
# de Lax-Friedrich (ver notas de relatividad numérica de Alcubierre).
function SolverMetric(Nr, dr, r, f1, f2, g1, g2)

    # Necesitamos una contribución de derivadas.
    juja = zeros(Nr)

    for j in 3:Nr-1
        juja[j] = f1[j]*(g2[j+1] - g2[j-1])/(2*dr) -
                f2[j]*(g1[j+1] - g1[j-1])/(2*dr) +
                g1[j]*(f2[j+1] - f2[j-1])/(2*dr) -
                g2[j]*(f1[j+1] - f1[j-1])/(2*dr)
    end

    juja[2]  = f1[2]*(-g2[4] + 4*g2[3] - 3*g2[2])/(2*dr) -
                f2[2]*(-g1[4] + 4*g1[3] - 3*g1[2])/(2*dr) +
                g1[2]*(-f2[4] + 4*f2[3] - 3*f2[2])/(2*dr) -
                g2[2]*(-f1[4] + 4*f1[3] - 3*f1[2])/(2*dr)
    
    juja[Nr] = f1[Nr]*(3*g2[Nr] - 4*g2[Nr-1] + g2[Nr-2])/(2*dr) -
                f2[Nr]*(3*g1[Nr] - 4*g1[Nr-1] + g1[Nr-2])/(2*dr) +
                g1[Nr]*(3*f2[Nr] - 4*f2[Nr-1] + f2[Nr-2])/(2*dr) -
                g2[Nr]*(3*f1[Nr] - 4*f1[Nr-1] + f1[Nr-2])/(2*dr)

    #------------------------------------------------
    # Solucionamos primero para a.
    a = zeros(Nr)
    a[2] = 1

    for j in 2:Nr-1
        
        rho = ( 2/( (r[j]*a[j])^2 ) )*( a[j]*( f1[j]^2 + f2[j]^2 - g1[j]^2 - g2[j]^2 ) + 
            ( 2*a[j]/r[j] )*( f1[j]*g2[j] - f2[j]*g1[j] ) +
            juja[j] )
        
        geo = ( a[j]^2 - 1 )/( 2*r[j] )

        aux_p = r[j]*( a[j]^2 )*rho - geo

        k1  = a[j]*aux_p

        x   = r[j] + dr/2
        y   = a[j] + k1*dr/2

        #------------------------------------------------
        rho = ( 2/( (x*y)^2 ) )*( y*( f1[j+1]^2 + f2[j+1]^2 - g1[j+1]^2 - g2[j+1]^2 ) +
                ( 2*y/x )*( f1[j+1]*g2[j+1] - f2[j+1]*g1[j+1] ) + 
                juja[j+1] )

        geo = ( y^2 - 1 )/( 2*x )

        aux = x*( y^2 )*rho - geo

        k2  = y*( aux + aux_p )/2

        #------------------------------------------------
        a[j+1] = a[j] + k2*dr

    end

    # Imponemos la simetría en el origen.
    a[1] = a[2]

    #------------------------------------------------
    # Ahora solucionamos para alpha.
    # Necesitamos solucionar hacia atrás porque conocemos una condición
    # en la frontera exterior, no en el inicio.
    α = zeros(Nr)
    α[Nr] = 1/a[Nr]

    for j in Nr-1:-1:2

        Srr = ( 2/( (r[j+1]*a[j+1])^2 ) )*juja[j+1]

        geo = ( a[j+1]^2 - 1 )/( 2*r[j+1] )

        aux_p = r[j+1]*(a[j+1]^2)*Srr + geo

        k1  = α[j+1]*aux_p

        x  = r[j+1] - dr/2
        y  = α[j+1] - k1*dr/2

        #------------------------------------------------
        Srr = ( 2/( (x*a[j])^2 ) )*juja[j]

        geo = ( a[j]^2 - 1 )/( 2*x )

        aux = x*(a[j]^2)*Srr + geo

        k2 = y*( aux + aux_p )/2

        #------------------------------------------------
        α[j] = α[j+1] - k2*dr
        
    end

    # Imponemos la simetría en el origen.
    α[1] = α[2]
    
    return [a, α]
end

In [ ]:
# Definimos una función para el ICN en sí.
function icn(Nr, dr, dt, r, a, α, f1, f2, g1, g2)

    # Limpiamos adot.
    adot = zeros(Nr)

    # Guardamos la información previa.
    a_p  = copy(a)
    α_p  = copy(α)
    f1_p = copy(f1)
    f2_p = copy(f2)
    g1_p = copy(g1)
    g2_p = copy(g2)
    
    #------------------------------------------------
    # Iniciamos el ciclo interno del ICN.
    for k in 1:3

        # Calculamos el tamaño del paso temporal en función de la iteración.
        if k < 3
            dtw = dt/2
        else
            dtw = dt
        end

        # Calculamos los lados derechos.
        fuentes = rhs(Nr, dr, r, a, α, f1, f2, g1, g2)
        f1_f = fuentes[1]
        f2_f = fuentes[2]
        g1_f = fuentes[3]
        g2_f = fuentes[4]

        # Actualizamos las funciones.
        for j in 2:Nr-1
            f1[j] = f1_p[j] + dtw*f1_f[j]
            f2[j] = f2_p[j] + dtw*f2_f[j]
            g1[j] = g1_p[j] + dtw*g1_f[j]
            g2[j] = g2_p[j] + dtw*g2_f[j]
        end
        
        # Imponemos las condiciones en la frontera interior.
        f1[1] = -f1[2]
        f2[1] = -f2[2]
        g1[1] = +g1[2]
        g2[1] = +g2[2]

        # Imponemos las condiciones en la frontera exterior ((59) de Daka).
        f1[Nr] = f1_p[Nr-1] + ((dr - dtw)/(dr + dtw))*(f1_p[Nr] - f1[Nr-1])
        f2[Nr] = f2_p[Nr-1] + ((dr - dtw)/(dr + dtw))*(f2_p[Nr] - f2[Nr-1])
        g1[Nr] = g1_p[Nr-1] + ((dr - dtw)/(dr + dtw))*(g1_p[Nr] - g1[Nr-1])
        g2[Nr] = g2_p[Nr-1] + ((dr - dtw)/(dr + dtw))*(g2_p[Nr] - g2[Nr-1])

        # Solucionamos la métrica.
        metricas = SolverMetric(Nr, dr, r, f1, f2, g1, g2)
        a = metricas[1]
        α = metricas[2]        

    #------------------------------------------------
    # Terminamos el ciclo interno.
    end

    # Calculamos la masa y la métrica.
    m = (r ./ 2) .* ( 1 .- (1 ./ a .^2) )
    σ = a .* α

    # Calculamos adot.
    adot[1] = 0
    adot[2] = 0
        
    for j in 3:Nr-1
        Srr = (2 / (r[j]^2 * a[j]) )*( 
                f1[j]*(f2[j+1] - f2[j-1]) -
                f2[j]*(f1[j+1] - f1[j-1]) +
                g1[j]*(g2[j+1] - g2[j-1]) -
                g2[j]*(g1[j+1] - g1[j-1]) )/(2*dr)
        
        Srr_p = (2 / (r[j]^2 * a_p[j]) )*( 
                f1_p[j]*(f2_p[j+1] - f2_p[j-1]) -
                f2_p[j]*(f1_p[j+1] - f1_p[j-1]) +
                g1_p[j]*(g2_p[j+1] - g2_p[j-1]) -
                g2_p[j]*(g1_p[j+1] - g1_p[j-1]) )/(2*dr)
        
        adot[j] = ( a[j] - a_p[j] )/dt +
                (r[j]/2)*( a[j]*α[j]*Srr + a_p[j]*α_p[j]*Srr_p )
    end
        
    adot[Nr] = 0
    
    return [m, a, α, σ, f1, f2, g1, g2, adot]
end

In [ ]:
# Definimos la función principal.
function main(f1, w, rf, dr, tf, courant)
    
    # Definimos la longitud de los arreglos.
    # (Ver las notas de relatividad numérica de Alcubierre).
    dt = courant*dr
    Nr = round(Int, rf/dr)
    Nt = round(Int, tf/dt) # tf*(Nr/(courant*rf))

    # Calculamos la condición de guardado temporal.
    savedataT = round(Int, Nt/100)
    savedataR = round(Int, Nr/100)

    # Definimos arreglos que guardarán la información.
    adot_dat = []
    f1_dat = []

    # Iniciamos el tiempo en cero.
    t = 0
    
    # Calculamos la condición inicial (la primer línea temporal).
    iniciales = initial(fi, w, Nr, dr)
    r  = iniciales[1]
    m  = iniciales[2]
    a  = iniciales[3]
    α  = iniciales[4]
    σ  = iniciales[5]
    f1 = iniciales[6]
    f2 = iniciales[7]
    g1 = iniciales[8]
    g2 = iniciales[9]
    adot = zeros(Nr)

    # Guardamos las condiciones iniciales.
    for j in 2:savedataR:Nr
        push!(adot_dat, [t, r[j], adot[j]])
    end
    push!(f1_dat, [t, f1[2]])

    # Graficamos las condiciones iniciales.
    grafica = plot(layout = (1, 3), size = (1500, 450))

    plot!(grafica[1], r, adot, label = "adot(r)", linecolor=:teal)
    
    plot!(grafica[2], r, a, label= "a(r)", linecolor=:magenta)
    plot!(grafica[2], r, σ, label= "σ(r)", linecolor=:sienna)
    plot!(grafica[2], r, α, label= "α(r)", linecolor=:teal)

    plot!(grafica[3], r, f1, label = "f1(r)", linecolor=:magenta)
    plot!(grafica[3], r, f2, label = "f2(r)", linecolor=:teal)

    # Iniciamos el ciclo principal de evolución
    # para calcular las demás líneas temporales.
    for n in 1:Nt

        # Avanzamos en el tiempo.
        t = t + dt

        # Hacemos el ICN.
        lineaTemporal = icn(Nr, dr, dt, r, a, α, f1, f2, g1, g2)
        m  = lineaTemporal[1]
        a  = lineaTemporal[2]
        α  = lineaTemporal[3]
        σ  = lineaTemporal[4]
        f1 = lineaTemporal[5]
        f2 = lineaTemporal[6]
        g1 = lineaTemporal[7]
        g2 = lineaTemporal[8]
        adot = lineaTemporal[9]

        # Guardamos información.
        if n % savedataT == 0

            # Guardamos información.
            for j in 2:savedataR:Nr
                push!(adot_dat, [t, r[j], adot[j]])
            end
            push!(f1_dat, [t, f1[2]])

            # Graficamos.
            plot!(grafica[1], r, adot, label="", linecolor=:teal)
    
            plot!(grafica[2], r, a, label="", linecolor=:magenta)
            plot!(grafica[2], r, σ, label="", linecolor=:sienna)
            plot!(grafica[2], r, α, label="", linecolor=:teal)
        
            plot!(grafica[3], r, f1, label="", linecolor=:magenta)
            plot!(grafica[3], r, f2, label="", linecolor=:teal)
        end

    end

    # Mostramos la gráfica.
    display(grafica)

    # Escribimos los archivos en sí.
    writedlm("adot.dat", adot_dat)
    writedlm("f1.dat", f1_dat)
    
    return
end

In [ ]:
# Definimos la condición de ejecución.
@time begin
    fi = 0.01              # Parámetro inicial del campo.
    w  = 1.0092992         # Ansatz de la Frecuencia de oscilación.
    rf = 95                # Radio de integración final.
    dr = 0.01              # Tamaño del paso (radial).
    tf = 10                # Tiempo final de integración.
    courant = 0.5          # parámetro de Courant.
        
    main(fi, w, rf, dr, tf, courant)
end